# Import Libraries

In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import math

# Load Data

### Import Load Data Functions

In [2]:
from LoadData import *

### Load Data and Preprocess

In [3]:
# load images
train_images, train_labels = load_data('training-a', 'training-a.csv')
train_images, train_labels = preprocess_data(train_images, train_labels)

# print shapes
print("train_images.shape = ", train_images.shape)
print("train_labels.shape = ", train_labels.shape)
print("train_images[0].shape = ", train_images[0].shape)

train_images.shape =  (1000, 28, 28, 1)
train_labels.shape =  (1000,)
train_images[0].shape =  (28, 28, 1)


### Split data into train and validation

In [4]:
# suffle data
s = np.arange(train_images.shape[0])
np.random.shuffle(s)
train_images = train_images[s]
train_labels = train_labels[s]

# split data into train and validation
train_ratio = 0.8
X_train = train_images[:int(train_ratio*len(train_images))]
y_train = train_labels[:int(train_ratio*len(train_labels))]
X_val = train_images[int(train_ratio*len(train_images)):]
y_val = train_labels[int(train_ratio*len(train_labels)):]

# Model

### Import Layers

In [5]:
from ConvolutionLayer import ConvolutionLayer
from ReLULayer import ReLULayer
from MaxPoolingLayer import MaxPoolingLayer
from FlattenLayer import FlattenLayer
from FullyConnectedLayer import FullyConnectedLayer
from SoftmaxLayer import SoftmaxLayer

### Import Model Class

In [6]:
from Model import Model

### Model Building

In [7]:
# use lenet-5 model
model = Model(10)
model.add(ConvolutionLayer(6, 5, 1, 1))
model.add(ReLULayer())
model.add(MaxPoolingLayer(pool_size=2, stride=2))
model.add(ConvolutionLayer(16, 5, 1, 1))
model.add(ReLULayer())
model.add(MaxPoolingLayer(pool_size=2, stride=2))
model.add(FlattenLayer())
model.add(FullyConnectedLayer(output_size=120))
model.add(ReLULayer())
model.add(FullyConnectedLayer(output_size=84))
model.add(ReLULayer())
model.add(FullyConnectedLayer(output_size=10))
model.add(SoftmaxLayer())

### Train Model

In [8]:
# train
model.train(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, learning_rate=0.00001, epochs=100, batch_size=64)

epoch:  0
y_pred (first 10)	:  [4 2 1 7 8 8 4 4 4 5]
y_real (first 10)	:  [5 2 2 8 1 8 5 5 5 5]
accuracy:  0.21
epoch:  1
y_pred (first 10)	:  [4 2 1 0 1 3 6 4 4 4]
y_real (first 10)	:  [5 2 2 8 1 8 5 5 5 5]
accuracy:  0.265
epoch:  2
y_pred (first 10)	:  [4 0 1 0 0 6 5 8 5 4]
y_real (first 10)	:  [5 2 2 8 1 8 5 5 5 5]
accuracy:  0.355
epoch:  3
y_pred (first 10)	:  [4 0 1 0 2 8 4 8 5 5]
y_real (first 10)	:  [5 2 2 8 1 8 5 5 5 5]
accuracy:  0.425
epoch:  4
y_pred (first 10)	:  [4 0 1 0 1 3 6 4 5 5]
y_real (first 10)	:  [5 2 2 8 1 8 5 5 5 5]
accuracy:  0.445
epoch:  5
y_pred (first 10)	:  [4 0 1 0 1 3 4 8 5 5]
y_real (first 10)	:  [5 2 2 8 1 8 5 5 5 5]
accuracy:  0.455
epoch:  6
y_pred (first 10)	:  [4 0 9 0 2 3 6 4 5 4]
y_real (first 10)	:  [5 2 2 8 1 8 5 5 5 5]
accuracy:  0.45
epoch:  7
y_pred (first 10)	:  [4 0 1 0 0 8 6 6 5 4]
y_real (first 10)	:  [5 2 2 8 1 8 5 5 5 5]
accuracy:  0.52
epoch:  8
y_pred (first 10)	:  [4 9 1 0 9 6 5 6 5 5]
y_real (first 10)	:  [5 2 2 8 1 8 5 5 5 5]
acc

KeyboardInterrupt: 

### Save Model

In [ ]:
# Save the model using pickle
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Load the model using pickle
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)


## Performace Metrics and Confusion Matrix

### Load Test Data

In [ ]:
# load test images
test_images, test_labels = load_data('training-d', 'training-d.csv')
test_images, test_labels = preprocess_data(test_images, test_labels)

### Performance Matrics

In [ ]:
# performance metrics and confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# predict   
y_pred = model.predict(test_images)

# print performance metrics
print("Accuracy: ", accuracy_score(test_labels, y_pred))
print("Confusion Matrix: ", confusion_matrix(test_labels, y_pred))

# print classification report
print("Classification Report: ", classification_report(test_labels, y_pred))

### Confustion Matrix

In [ ]:
# plot confusion matrix using seaborn
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(test_labels, y_pred)
plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt="d")
plt.title("Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()